In [ ]:
#import des librairies
from collatex import *
from lxml import etree
import os
import json
#ici, on a stocké les fonctions dans un autre fichier, qu'on appelle
from fonctions import *

#le script crée un dossier 'sorties' avec des sous-dossiers contenant les tables et aussi les résultats par Div
#le fichier xml final se place également dans ce dossier

In [ ]:
#création des témoins : on les parse pour récupérer
A1 = etree.parse('textes/A.xml')
B1 = etree.parse('textes/B.xml')

In [ ]:
v = []
vDiv = []
propreId = 1

#il s'agit de la même boucle que précédemment, mais ici on a une surstructuration de div
#les div sont liées par leurs attributs @xml:id et @corresp

#pour chaque div
for div in A1.xpath("/text/div"):
    #on récupère l'id
    iDiv = div.get('{http://www.w3.org/XML/1998/namespace}id')
    print(iDiv)
    #pour chaque div telle que cela correspond à l'id
    for divEz in B1.xpath("/text/div[@corresp='"+iDiv+"']"):
        iDivEz = divEz.get('{http://www.w3.org/XML/1998/namespace}id')
    for par in A1.xpath("/text/div[@xml:id='"+iDiv+"']/p"):
        #on récupère le numéro de paragraphe
        i = par.get('n')
        # on vérifie que tout se passe bien
        print(i)
        #A = le paragraphe en question
        A = par
        print(A)
        #on récupère aussi le paragraphe qui correspond en Ez (B)  
        for p in B1.xpath("/text/div[@xml:id='"+iDivEz+"']/p[@n='"+i+"']"):
        # B = le paragraphe qui porte le même numéro que en A
            B = p
            print(B)
        
        json_input = {}
        json_input['witnesses'] = []
        json_input['witnesses'].append(XMLtoJson('A',A))
        json_input['witnesses'].append(XMLtoJson('B',B))
        print(json_input)
        
        #création de l'objet collation
        json_collation = Collation()
        for witness in json_input["witnesses"]:
            json_collation.add_witness(witness)
        collation = json_collation
        print(collation)
        #vérif XML
        coll = collate(collation, output="xml",segmentation=False)
        #collation
        table = collate(collation, output="table",segmentation=False)
        #export spécifique pour avoir des belles tables
        t = notretable(table)
        #on transforme la table sous notre format en chaîne de caractères, qu'on écrite ensuite dans un fichier : attention : 123 fichiers
        tables = str(t)
        if not os.path.isdir("sorties"):
            os.makedirs("sorties")
        if not os.path.isdir("./sorties/tables"):
            os.makedirs("./sorties/tables")
        with open("./sorties/tables/table_"+iDiv[3:6]+"_"+i, "w") as w:
            w.write(tables)
        result = notre_export_xml(table)
        result    

    #création d'un objet dans lequel on stocke notre résultat, à l'intérieur d'un paragraphe qui se définit comme tel
        doc = '<p n="' + i +'">' + result + '</p>'
        v.append(doc)
    #on sort de la boucle sur les paragraphes    
    #ajout pour réunir les div    
    valDiv = '<div xml:id="divColl' + str(propreId) +'" corresp="' + iDiv +'_'+ iDivEz +'">'+''.join(v) +  '</div>'
    if not os.path.isdir("./sorties/collParDiv"):
        os.makedirs("./sorties/collParDiv")
    #on va écrire chaque div dans un fichier séparé en plus    
    with open("./sorties/collParDiv/collationParDiv"+str(propreId)+".xml", "w") as text_fileDiv:
        text_fileDiv.write(valDiv)
    vDiv.append(valDiv)
    propreId+=1
    v = []        

In [ ]:
#on crée un objet dans lequel on place notre liste, à l'intérieur d'une balise racine <text>
val = "<text>" + ''.join(vDiv) + "</text>"

In [ ]:
#on écrit le résultat (une simple chaîne de caractères) dans notre document XML
with open("./sorties/resultat_collation_divEtPar.xml", "w") as text_file:
    text_file.write(val)

In [ ]:
#on peut regarder ce que donne l'export table
table = collate(collation, output="table",segmentation=False)
t = notretable(table)
t